<a href="https://colab.research.google.com/github/jorcisai/APR/blob/master/neuralnets/src/mnist_local_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST loader

Importing standard packages and tensorflow_datasets to ease data manipulation.

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf
import os

import tensorflow_datasets as tfds

First, you need to upload mnist_dataset.py and original MNIST data files from your drive

In [ ]:
from google.colab import files

uploaded = files.upload()

Loading training and test images and labs from array into dataset object. Then, creating pairs of (image,label)

In [27]:
from mnist_dataset import get_mnist

(X, xl), (Y, yl) = get_mnist("./").load_data()
N = X.shape[0]
C = len(set(xl))
Xdat   = tf.data.Dataset.from_tensor_slices(X)
xldat  = tf.data.Dataset.from_tensor_slices(xl)
Xxldat = tf.data.Dataset.zip((Xdat, xldat))
Ydat   = tf.data.Dataset.from_tensor_slices(Y)
yldat  = tf.data.Dataset.from_tensor_slices(yl)
Yyldat = tf.data.Dataset.zip((Ydat, yldat))

Taking a look at a couple of training images and labels after being converted into dataset type.

In [ ]:
print(N,C)
for (image,label) in Xxldat.take(1):
  print(label)
  print(image)

# Experimental design

Use tf.data.Dataset.take and tf.data.Dataset.skip to split training dataset into 90% for training and 10% for dev.

Before being passed into the model, the datasets need to be shuffled and batched. So, first, the complete dataset is shuffled with a fixed seed so that we can repeat the same shuffle of the dataset, then the dataset is split into training, validation and test, and each of these subsets is batched.

In [42]:
Ntr = round(N*0.9)
Ndv = N-Ntr
Nbatch = 1000

Xxldat = Xxldat.shuffle(N,seed=13)

Xxltr = Xxldat.take(Ntr)
Xxltr = Xxltr.batch(Nbatch)
Xxldv = Xxldat.skip(Ntr).take(Ndv)
Xxldv = Xxldv.batch(Nbatch)
Yylte = Yyldat.batch(Nbatch)

# Build the model

Create an empty model and add layers to it.

In [60]:
model = tf.keras.Sequential()

We'll have a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.

In [61]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [800]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(C, activation='softmax'))

Finally, compile the model. For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. You can try other optimizers, but adam is very common.

In [62]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

This model running for 10 epochs on this data produces decent results

In [63]:
model.fit(Xxltr, epochs=10, validation_data=Xxldv)

Epoch 1/20
54/54 [==============================] - 4s 65ms/step - loss: 23.8135 - accuracy: 0.8162 - val_loss: 3.2008 - val_accuracy: 0.9257
Epoch 2/20
54/54 [==============================] - 3s 63ms/step - loss: 2.1980 - accuracy: 0.9366 - val_loss: 1.1313 - val_accuracy: 0.9533
Epoch 3/20
54/54 [==============================] - 3s 63ms/step - loss: 1.0259 - accuracy: 0.9544 - val_loss: 0.4965 - val_accuracy: 0.9667
Epoch 4/20
54/54 [==============================] - 3s 63ms/step - loss: 0.5837 - accuracy: 0.9662 - val_loss: 0.3667 - val_accuracy: 0.9748
Epoch 5/20
54/54 [==============================] - 3s 63ms/step - loss: 0.3818 - accuracy: 0.9734 - val_loss: 0.2775 - val_accuracy: 0.9810
Epoch 6/20
54/54 [==============================] - 3s 64ms/step - loss: 0.2557 - accuracy: 0.9795 - val_loss: 0.1454 - val_accuracy: 0.9868
Epoch 7/20
54/54 [==============================] - 3s 64ms/step - loss: 0.1693 - accuracy: 0.9838 - val_loss: 0.0944 - val_accuracy: 0.9887
Epoch 8/20
5

# Evaluate the model

Compute accuracy on the test set (accuracy 98.6%)

In [64]:
loss_te, acc_te = model.evaluate(Yylte)

print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(loss_te, acc_te))

10/10 [==============================] - 0s 29ms/step - loss: 0.6859 - accuracy: 0.9691

Eval loss: 0.686, Eval accuracy: 0.969
